In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator

Using Theano backend.


In [2]:
def createCNN(classes,cnnOutput=(32,3,3), poolSize=(2,2)):
    classifier = Sequential()
    classifier.add(Convolutional2D(cnnOutput[0],cnnOutput[1],cnnOutput[2], input_shape(64,64,3), activation='relu'))
    classifier.add(MaxPooling2D(pool_size=poolSize))
    classifier.add(Flatten())
    classifier.add(Dense(output_dim=classes, activation='relu'))
    classifier.add(Dense(output_dim=1, activation='sigmoid'))
    classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy'])
    
    return classifier

In [11]:
def sperateData():
    import os
    # for folder in os.listdir('images/train'):
    #     os.mkdir('images/test/'+folder)
    import shutil
    import numpy as np
    for folder in os.listdir('images/train'):
        for image in os.listdir('images/train/'+folder):
            if np.random.rand(1)<0.2:
                shutil.move('images/train/'+folder+'/'+image, 'images/test/'+folder+'/'+image)
for folder in os.listdir('images/train'):
    

In [ ]:
def getDataset(classes,train_source,test_source,save_dir):
    trainGen = ImageDataGenerator(
    rescale = 1.0/255,
    shear_range = 0.1,
    zoom_range = 0.2,
    horizontal_flip = True,
    vertical_flip=True
    )
    
    testGen = ImageDataGenerator(rescale = 1.0/255)
    
    trainSet = trainGen.flow_from_directory(train_source, target_size=(64,64), batch_size=32, class_mode='categorical',save_to_dir=save+'_gen')